# Perceptron
Implementation of the classic Perceptron by Frank Rosenblatt for binary classification.

## Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
%matplotlib inline

## Dataset

In [23]:
data = np.genfromtxt("./data/data.txt", delimiter="\t")
X, y = data[:, :2], data[:, 2]
y = y.astype("int")

print("Class counts:", np.bincount((y)))
print("X shape", X.shape)
print("y shape", y.shape)

# Shuffling & train/test split
shuffle_idx = np.arange(y.shape[0])
shuffle_rng = np.random.RandomState(1)
shuffle_rng.shuffle(shuffle_idx)
X, y = X[shuffle_idx], y[shuffle_idx]
X_train, X_test = X[shuffle_idx[:70]], X[shuffle_idx[70:]]
y_train, y_test = y[shuffle_idx[:70]], y[shuffle_idx[70:]]

# Normalize


Class counts: [50 50]
X shape (100, 2)
y shape (100,)
